In [1]:
import torch
from torchinfo import summary
from model import AttenResUnet

ures = AttenResUnet()

x = torch.randn(1, 3, 256, 256)  # Example input tensor
summary(ures, input_data=x, col_names=["input_size", "output_size", "num_params", "kernel_size"])

c:\Users\devya\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\devya\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
AttenResUnet                             [1, 3, 256, 256]          [1, 21, 256, 256]         --                        --
├─Sequential: 1-1                        [1, 3, 256, 256]          [1, 64, 128, 128]         --                        --
│    └─Conv2d: 2-1                       [1, 3, 256, 256]          [1, 64, 128, 128]         (9,408)                   [7, 7]
│    └─BatchNorm2d: 2-2                  [1, 64, 128, 128]         [1, 64, 128, 128]         (128)                     --
│    └─ReLU: 2-3                         [1, 64, 128, 128]         [1, 64, 128, 128]         --                        --
├─MaxPool2d: 1-2                         [1, 64, 128, 128]         [1, 64, 64, 64]           --                        3
├─Sequential: 1-3                        [1, 64, 64, 64]           [1, 64, 64, 64]           --                        --
│    └─Basi